extracting matrix from mne object, creating a log and calling function to find the pulses

In [ ]:
def main( ): #main arguments: raw, logs
    raw = mne.io.read_raw_brainvision('sub100_rt_TEP.vhdr')
    mat = raw.get_data()
    output_log = {}
    num_of_artifacts = {}
    logs = get_interpolation(raw)


In [ ]:
def get_interpolation(raw):
    logs = {}
    logs['Errors'] = []
    logs['General Info'] = {}
    mat = raw.get_data()  # create 65 row matrix with ~7200000. Row = brain area, Column = 0.1 ms
    diffs = np.diff(mat)
    peaks = get_peak(mat)

determine abnormal values in differences matrix

In [ ]:
def get_peak(mat):
    max_mat = np.amax(mat, axis=1)
    min_mat = np.amin(mat, axis=1)
    peaks = (max_mat + np.abs(min_mat)) / 7
    return peaks


iterating over matrix's rows, for each row, apply all of the followings: 

In [ ]:
for i in range(len(mat)):  # for each region
        peak = peaks[i]  # get_peak(max_mat[i], min_mat[i])  #determine required amplitude
        logs['Region ' + str(i + 1)] = {'Amplitude': peak}
        indices = find_abnormalities(diffs[i], peak, i)  # find abnormal indices
        pulses_lst = group_indices(indices, 6)  # group abnormal indices by adjacency
        sort_pulses_for_region(pulses_lst)  # sort indices
        final_array = check_shape(mat[i], pulses_lst, peak, i)  # check pattern of pulses contains min&max values
        logs['Region ' + str(i + 1)]['Number of pulses'] = len(final_array)
        logs['Region ' + str(i + 1)]['Indices'] = final_array
    lst = find_range(logs, len(mat), logs['Region 1']['Number of pulses'])  # determine final locations of pulses
    logs['General Info']["Indices to interpolate"] = lst
    return logs

Using the value from get_peaks, we find abnormal indices

In [ ]:
def find_abnormalities(diffs, peak, j):
    low_inds = np.where(diffs > peak)
    high_inds = np.where(diffs < (-peak))
    return (high_inds, low_inds)


Because each pulse contains several abnormal indices, we will group all indices that are at most 0.6ms apart into one pulse 

In [ ]:
def group_indices(indices, freq):
    high_inds = indices[0][0]
    low_inds = indices[1][0]
    pulses_lst = [[]]  # initiate grouped abnormal indices
    group_first_by_low(low_inds, pulses_lst, freq)
    add_high_to_groups(high_inds, pulses_lst, freq)
    return pulses_lst

example of grouping abnormal low values

In [ ]:
def group_first_by_low(low_inds, pulses_lst, freq):  # creates a nested list for each suspected pulse
    cnt = 0
    for i in range(len(low_inds)):  # Loop to group values to pulses
        pulses_lst[cnt].append(low_inds[i])
        try:  # avoid error incase we reached end of list indices
            if abs(low_inds[i] - low_inds[i + 1]) > freq:  # If next index is more than 0.6 ms apart - NOT the same pulse
                cnt += 1
                pulses_lst.append([])  # Add another row for the next pulse of this area
        except IndexError:
            cnt += 1
            pass
    return

Sort all indices within each pulse after high and low grouping

In [ ]:
def sort_pulses_for_region(pulses_lst):
    for i in range(len(pulses_lst)):
        pulses_lst[i] = np.sort(pulses_lst[i])

for each pulse we make sure it contains at least one abnormal high value and at least one abnormal low value - to match the required pattern of TMS pulse

In [ ]:
def check_shape(row, pulses_lst, peak, j):
    final_array = []
    for i in range(len(pulses_lst)):
        if len(pulses_lst[i]) == 0:
            continue
        values = row[pulses_lst[i]]
        inds = [k for k in range(np.amin(pulses_lst[i] - 10), np.amax(pulses_lst[i] + 10))]
        med = np.median(row[inds])
        if np.amin(values) < med - peak / 3 and np.amax(values) > med + peak / 3:
            final_array.append(pulses_lst[i].tolist())
    return final_array

for each pulse, we find the average pulse start point and the average pulse endpoint and create a 2ms segment that contains the pulse

In [ ]:
def find_range(logs, rows, pulses):  # take average of all indices and create a 2 ms section
    result = []
    for i in range(pulses):
        min_sum = 0
        max_sum = 0
        cnt = 0
        for j in range(rows):
            try:
                max_sum += logs['Region ' + str(j + 1)]['Indices'][i][-1]
                min_sum += logs['Region ' + str(j + 1)]['Indices'][i][0]
            except IndexError:
                cnt += 1
                logs['Errors'].append("Error in row " + str(j) + " In pulse " + str(i))
        max_avg = max_sum // (rows - cnt)
        min_avg = min_sum // (rows - cnt)
        diff = max_avg - min_avg
        extension = (20 - diff) // 2
        result.append([min_avg - extension, max_avg + extension])
    return result

In [ ]:
returning to main flow, we create a new matrix, then iterate over the rows

In [ ]:
interpolated_mat = np.zeros(shape = (len(mat), len(mat[0])))
for channel in range(len(mat)): #iterating over the channels
        key = "Region " + str(channel+1)
        output_log["sum_of_peaks"] += len(logs[key]["Indices"]) 
        xaxis , yaxis = interpolation_axises(mat[channel] , logs['General Info']["Indices to interpolate"] )
        interpolated_mat[channel] = interpolation(xaxis, yaxis, mat[channel])
        if len(mat[channel]) != len(interpolated_mat[channel]):
            print("error! interpolated row has different length  " + str(len(interpolated_mat[channel])) + " from original mat " + str(len(mat[channel])))
        else:
            count = 0
            avg = 0
            for n in range(len(xaxis)):
                i = xaxis[n]
                avg += abs(interpolated_mat[channel][i] - mat[channel][i])
                if mat[channel][i] != interpolated_mat[channel][i]:
                    count += 1
            print("found " + str(count) + " different values after interpolation")
            print("avg. difference between mats is " + str(avg/len(xaxis)))
        if channel == 0:
            plt.plot( [x for x in range(len(interpolated_mat[channel]))] , interpolated_mat[channel] , label = "interpolated")
            plt.scatter(xaxis , yaxis , label = "original" , color = 'red')
            plt.legend()
            plt.show()

In [ ]:
we create 

In [ ]:
def interpolation_axises(row , indices):
    xaxis = [ i for i in range(indices[0][0])]
    for index in range(len(indices)):
        try : 
            xaxis += [i for i in range(indices[index][1] , indices[index + 1][0])]
        except IndexError:
            xaxis += [i for i in range(indices[index][1] , len(row))]  
    yaxis = [row[i] for i in xaxis] 
    return xaxis , yaxis